In [1]:
import pandas as pd
import numpy as np

# (1) Data Preprocessing

먼저 manual하게 데이터 중에서 이름이 Hong_Kong,China 이런식으로 되어 있는 애들은 delimeter를 ,로 해서 csv 파일을 읽어올 때 오류를 일으키기 때문에 manual 하게 없애주었다. Macau_,China랑 2가지가 문제였다. 

In [2]:
def preprocessing():
    # 데이터 로드
    DP01 = pd.read_csv('data/wmo_normals_9120_DP01.csv')
    MNVP = pd.read_csv('data/wmo_normals_9120_MNVP.csv')
    MSLP = pd.read_csv('data/wmo_normals_9120_MSLP.csv')
    PRCP = pd.read_csv('data/wmo_normals_9120_PRCP.csv')
    TAVG = pd.read_csv('data/wmo_normals_9120_TAVG.csv')
    TMAX = pd.read_csv('data/wmo_normals_9120_TMAX.csv')
    TMIN = pd.read_csv('data/wmo_normals_9120_TMIN.csv')
    TSUN = pd.read_csv('data/wmo_normals_9120_TSUN.csv')

    # 데이터 전처리
    datasets = [DP01, MNVP, MSLP, PRCP, TAVG, TMAX, TMIN, TSUN]
    param_map = {
        1: 'Precipitation (mm)',
        2: 'Number of Days with Precipitation ≥ 1 mm (#Days)',
        3: 'Mean Daily Maximum Temperature (degC)',
        4: 'Mean Daily Minimum Temperature (degC)',
        5: 'Mean Daily Mean Temperature (degC)',
        6: 'Mean Sea Level Pressure (hPa)',
        7: 'Mean Vapor Pressure (hPa)',
        8: 'Total Number of Hours of Sunshine (Hours)'
    }

    def clean_columns(df):
        df.columns = df.columns.str.strip()
        return df

    datasets = [clean_columns(df) for df in datasets]
    all_data = pd.concat(datasets, ignore_index=True)
    all_data = all_data.drop(columns=['Rgn', 'ID', 'WIGOS_ID', 'Elevation', 'Annual'])

    # Parameter 컬럼 추가
    all_data['Parameter'] = all_data['Elem'].map(param_map)

    # 데이터 앞뒤 공백 제거
    all_data = all_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)

    # 데이터가 -99.9인 경우는 값이 없는 경우이므로 NaN으로 바꿈.
    all_data = all_data.replace(-99.9, np.nan)

    # nan 값이 있는 행 제거
    all_data = all_data.dropna()
    
    # Save the all_data DataFrame to a new CSV file
    all_data.to_csv('data.csv', index=False)

    return all_data

# Call the preprocessing function
df = preprocessing()

/tmp/ipykernel_2799374/2069334829.py:37: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  all_data = all_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [3]:
df.shape

(36742, 18)

In [4]:
df.columns

Index(['Elem', 'Latitude', 'Longitude', 'Country', 'Station', 'Jan', 'Feb',
       'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec',
       'Parameter'],
      dtype='object')

In [5]:
df.head()

,Elem,Latitude,Longitude,Country,Station,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Parameter
0,2,27.837,-0.194,Algeria,Adrar,0.4,0.3,0.3,0.3,0.2,0.1,0.1,0.3,0.5,0.4,0.4,0.1,Number of Days with Precipitation ≥ 1 mm (#Days)
1,2,32.752,-0.594,Algeria,AinSefra,2.8,2.4,3.1,2.5,2.1,1.4,1.1,2.9,3.7,3.2,2.9,2.3,Number of Days with Precipitation ≥ 1 mm (#Days)
2,2,36.690,3.217,Algeria,AlgerDarElBeida,8.4,8.7,7.0,6.1,4.3,1.4,0.4,1.4,4.0,5.4,9.2,8.2,Number of Days with Precipitation ≥ 1 mm (#Days)
3,2,36.822,7.803,Algeria,Annaba,10.3,10.2,8.2,7.0,4.7,2.0,0.6,1.8,6.2,7.2,10.0,10.8,Number of Days with Precipitation ≥ 1 mm (#Days)
4,2,35.761,6.320,Algeria,Batna,5.1,4.6,5.5,5.2,5.1,2.2,1.7,3.0,4.6,4.2,4.3,5.2,Number of Days with Precipitation ≥ 1 mm (#Days)


# (2) 본격적으로 구현!

In [6]:
import pandas as pd
import altair as alt
from ipywidgets import interact, widgets
from vega_datasets import data
import pandas as pd
import altair as alt
from vega_datasets import data
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import Output

# Altair 데이터 변환 설정
alt.data_transformers.disable_max_rows()

param_map = {
    1: 'Precipitation (mm)',
    2: 'Number of Days with Precipitation ≥ 1 mm (#Days)',
    3: 'Mean Daily Maximum Temperature (degC)',
    4: 'Mean Daily Minimum Temperature (degC)',
    5: 'Mean Daily Mean Temperature (degC)',
    6: 'Mean Sea Level Pressure (hPa)',
    7: 'Mean Vapor Pressure (hPa)',
    8: 'Total Number of Hours of Sunshine (Hours)'
}

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

In [ ]:
# Altair 지도 데이터
world_map = alt.topo_feature(data.world_110m.url, feature="countries")

def plot_heatmap(selected_param, selected_month):
    # 선택된 파라미터와 월 기준으로 데이터 필터링
    filtered = df[df['Parameter'] == selected_param]

    # Heatmap 생성
    background = alt.Chart(world_map).mark_geoshape(
        fill='lightgray',
        stroke='white'
    ).project('naturalEarth1').properties(
        width=800,
        height=400
    )

    heatmap = alt.Chart(filtered).mark_circle().encode(
        longitude='Longitude:Q',
        latitude='Latitude:Q',
        color=alt.Color(f'{selected_month}:Q', scale=alt.Scale(scheme='viridis')),
        size=alt.Size(f'{selected_month}:Q', title='Intensity'),
        tooltip=[
            'Station:N',                   # 지점 이름
            'Country:N',                   # 국가 이름
            'Latitude:Q',                  # 위도 추가
            'Longitude:Q',                 # 경도 추가
            alt.Tooltip(f'{selected_month}:Q', title=selected_param)  # 선택된 파라미터 값
        ]
    )

    return background + heatmap

# 파라미터 선택 위젯
param_dropdown = widgets.Dropdown(
    options=list(param_map.values()),
    value='Precipitation (mm)',
    description='Parameter:'
)

# 월 선택 위젯
month_dropdown = widgets.Dropdown(
    options=months,
    value='Jan',
    description='Month:'
)

def update(selected_param, selected_month):
    chart = plot_heatmap(selected_param, selected_month)
    chart.display()

interact(
    update,
    selected_param=param_dropdown,
    selected_month=month_dropdown,
)

interactive(children=(Dropdown(description='Parameter:', options=('Precipitation (mm)', 'Number of Days with P…

<function __main__.update(selected_param, selected_month)>

In [10]:
# 필터링 슬라이더
slider = widgets.FloatRangeSlider(
    value=[0, 100],
    min=0,
    max=500,
    step=1,
    description='Filter:',
    layout=widgets.Layout(width='80%')
)

# Output 위젯
output = Output()

# 최소/최대값 계산 함수
def calculate_min_max(selected_param, selected_month):
    filtered = df[df['Parameter'] == selected_param]
    min_val = filtered[selected_month].min()
    max_val = filtered[selected_month].max()
    return min_val, max_val

# 슬라이더 업데이트 함수
def update_slider(selected_param, selected_month):
    min_val, max_val = calculate_min_max(selected_param, selected_month)
    slider.min = min_val
    slider.max = max_val
    slider.value = [min_val, max_val]

# 깃발 추가 함수
def add_flags(selected_param, selected_month, value_range):
    min_val, max_val = value_range
    filtered = df[
        (df['Parameter'] == selected_param) &
        (df[selected_month] >= min_val) &
        (df[selected_month] <= max_val)
    ]

    points = alt.Chart(filtered).mark_text(text="🚩", size=10, color='red').encode(
        longitude='Longitude:Q',
        latitude='Latitude:Q',
        tooltip=[
            'Station:N',                   # 지점 이름
            'Country:N',                   # 국가 이름
            'Latitude:Q',                  # 위도 추가
            'Longitude:Q',                 # 경도 추가
            alt.Tooltip(f'{selected_month}:Q', title=selected_param)  # 선택된 파라미터 값
        ]
    )

    return points

# 업데이트 함수
def update_plot(change=None):
    with output:
        # Output 위젯 초기화
        output.clear_output()
        
        # 현재 선택 상태
        selected_param = param_dropdown.value
        selected_month = month_dropdown.value
        value_range = slider.value

        # Heatmap 생성
        heatmap_chart = plot_heatmap(selected_param, selected_month)

        # 깃발 추가
        flag_chart = add_flags(selected_param, selected_month, value_range)

        # 두 차트 결합
        chart = heatmap_chart + flag_chart

        # 차트 표시
        display(chart)

# 이벤트 핸들러 등록
param_dropdown.observe(update_plot, names='value')
month_dropdown.observe(update_plot, names='value')
slider.observe(update_plot, names='value')

# 슬라이더 동적 업데이트
def update_slider_on_change(change):
    selected_param = param_dropdown.value
    selected_month = month_dropdown.value
    update_slider(selected_param, selected_month)

param_dropdown.observe(update_slider_on_change, names='value')
month_dropdown.observe(update_slider_on_change, names='value')

# UI 표시
ui = widgets.VBox([param_dropdown, month_dropdown, slider])
display(ui, output)

# 초기 차트 표시
update_plot()

Output()

In [9]:
# Output 위젯
output_table = Output()

# 깃발이 꽂힌 나라와 지역을 표 형식으로 출력하는 함수
def display_flagged_locations(selected_param, selected_month, value_range):
    min_val, max_val = value_range
    filtered = df[
        (df['Parameter'] == selected_param) &
        (df[selected_month] >= min_val) &
        (df[selected_month] <= max_val)
    ]
    
    with output_table:
        output_table.clear_output()
        display(filtered[['Country', 'Station', 'Latitude', 'Longitude', selected_month]])

# 업데이트 함수에 표 출력 추가
def update_plot_with_table(change=None):
    with output:
        # Output 위젯 초기화
        output.clear_output()
        
        # 현재 선택 상태
        selected_param = param_dropdown.value
        selected_month = month_dropdown.value
        value_range = slider.value

        # Heatmap 생성
        heatmap_chart = plot_heatmap(selected_param, selected_month)

        # 깃발 추가
        flag_chart = add_flags(selected_param, selected_month, value_range)

        # 두 차트 결합
        chart = heatmap_chart + flag_chart

        # 차트 표시
        display(chart)
    
    # 깃발이 꽂힌 나라와 지역을 표 형식으로 출력
    display_flagged_locations(selected_param, selected_month, value_range)

# 이벤트 핸들러 등록
param_dropdown.observe(update_plot_with_table, names='value')
month_dropdown.observe(update_plot_with_table, names='value')
slider.observe(update_plot_with_table, names='value')

# UI 표시
ui_with_table = widgets.VBox([param_dropdown, month_dropdown, slider, output, output_table])
display(ui_with_table)

# 초기 차트 및 표 표시
update_plot_with_table()

내가 가진 데이터프레임 df에는 Index(['Elem', 'Latitude', 'Longitude', 'Country', 'Station', 'Jan', 'Feb',
       'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec',
       'Parameter'],
      dtype='object')라는 열들이 있어.

사용자가 먼저 month를 선택하면 데이터프레임에서 그 month에 대한 정보들만을 보는거야. 그래서 만약에 사용자가 맨처음에 3월을 골랐으면 데이터 프레임에서 Jan~Dec중에 Mar column의 값만 고려하는거지. 그리고 Parameter column에는 8개의 서로 다른 날씨에 대한 정보가 들어있어. param_map = {
    1: 'Precipitation (mm)',
    2: 'Number of Days with Precipitation ≥ 1 mm (#Days)',
    3: 'Mean Daily Maximum Temperature (degC)',
    4: 'Mean Daily Minimum Temperature (degC)',
    5: 'Mean Daily Mean Temperature (degC)',
    6: 'Mean Sea Level Pressure (hPa)',
    7: 'Mean Vapor Pressure (hPa)',
    8: 'Total Number of Hours of Sunshine (Hours)'
} 이렇게 8가지야.

그래서 만약에 Parameter가 Mean Vapor Pressure (hpa)인 행의 Mar column의 값은 3월의 Mean Vapor Pressure 값이야. 이런 상황에서 사용자가 슬라이더를 통해 각 Parameter의 범위를 설정하면, 데이터 중에서 그 달의 값이 그 범위 내에 속하는 데이터만 골라서 지도 위에 red flag로 표시하게 하고 싶어. 

모든 8개의 슬라이더 조건을 모두 만족하는 국가만 표시하고 지도 밑에 표로도 출력하게 하고 싶어.